In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
from glob import glob
from tqdm import tqdm

txts = glob('salina/*salina*/*.txt')
txts.extend(glob('../youtube/malay2/*salina-short*/*salina*/*.txt'))
len(txts)

39482

In [3]:
import malaya

In [4]:
model = malaya.true_case.transformer(model = 'small', quantized = True)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np

def compute_sparse_correlation_matrix(A):
    scaler = StandardScaler(with_mean = False)
    scaled_A = scaler.fit_transform(A)
    corr_matrix = (1 / scaled_A.shape[0]) * (scaled_A.T @ scaled_A)
    return corr_matrix

def filter_splitted(s, t, threshold = 0.001):
    bow = CountVectorizer(token_pattern = '[A-Za-z0-9]+').fit([s])
    s_bow = bow.transform([s] + t)
    score = np.array(compute_sparse_correlation_matrix(s_bow.T).todense())[0,1:]
    r = [t[no] for no, s in enumerate(score) if s >= threshold]
    return ' '.join(r)

In [8]:
results = []
batch_size = 1
for i in tqdm(range(0, len(txts), batch_size)):
    index = min(i + batch_size, len(txts))
    batch = txts[i: index]
    r = []
    for t in batch:
        with open(t) as fopen:
            r.append(fopen.read())
    
    c = model.true_case(r, beam_search = False)

    for k in range(len(batch)):
        s = r[k]
        t = malaya.text.function.split_into_sentences(c[k])
        l = len(malaya.text.function.split_into_sentences(s))
    
    results.extend(list(zip(batch, r, c)))

100%|██████████| 39482/39482 [1:46:18<00:00,  6.19it/s]  


In [9]:
results

[('salina/output-text-salina/tiga-4.mp3-441.wav.txt',
  'bila hendak ketawa berlanjutan memandang ibunya sambil berkata',
  'Bila hendak ketawa berlanjutan memandang ibunya sambil berkata,'),
 ('salina/output-text-salina/dua-puluh-enam-27.mp3-375.wav.txt',
  'dirasakannya Bahawa tidak ada gunanya kalau itu diceritakannya Biarpun Sunarto yang bertanya',
  'Dirasakannya bahawa tidak ada gunanya kalau itu diceritakannya biarpun Sunarto yang bertanya".'),
 ('salina/output-text-salina/dua-puluh-sembilan-30.mp3-233.wav.txt',
  'keluar dengan mukanya yang nampaknya begitu puas',
  'Keluar dengan mukanya yang nampaknya begitu puas.'),
 ('salina/output-text-salina/lapan-9.mp3-43.wav.txt',
  'orang bodoh sahaja Razman kurus tinggi orangnya',
  'Orang bodoh sahaja Razman kurus tinggi orangnya.'),
 ('salina/output-text-salina/sebelas-12.mp3-115.wav.txt',
  'apa tidak dapat dia hendak melupakan hal itu Walaupun dicubanya sungguh-sungguh dengan membaca buku sekolahnya',
  'Apa tidak dapat dia hendak

In [10]:
import json

with open('true-case-female.json', 'w') as fopen:
    json.dump(results, fopen)